In [1]:
from enum import Enum

class Workspace(Enum):
    py=1
    note=2
    
workspace = Workspace.note


In [2]:
import numpy as np
import os
import pandas as pd
from pylab import rcParams
from matplotlib import pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
import csv
import cv2
import time
from PIL import Image
from datetime import datetime
import random
import json

from sklearn.model_selection import train_test_split
import shutil

import imghdr


%matplotlib inline
register_matplotlib_converters()
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)



In [3]:
if workspace != Workspace.py:
    path = '.'
else:
    parser = optparse.OptionParser()
    
    parser.add_option('-p', '--path', help='Pass the path image', default='.')
    
    (opts, args) = parser.parse_args()  # instantiate parser
    
    path = opts.path
    new_dataset_path = f'{path}/new_dataset/'
    
    print(opts.path)


# In[2]:

path


'.'

In [4]:

os.system(f'tar -zxvf {path}/dataset_roboflow.tar.gz -C {path}')
os.system(f'tar -zxvf {path}/dataset_sketch_it.tar.gz -C {path}')

# !tar -zxvf dataset_roboflow.tar.gz && tar -zxvf dataset_sketch_it.tar.gz

dataset_roboflow/train/00021_png.rf.f33255cfe99ce33a5b61c360f67a8c58.jpg
dataset_roboflow/train/image_38_png.rf.e69ffcc3c101dd4cf1a1f50d8389da52.jpg
dataset_roboflow/valid/image_74_png.rf.52beed189736dbe45515e7820607e606.jpg
dataset_roboflow/new_train/00034_png.rf.cdaed64c2e2f8fe839439f6c387436e5.jpg
dataset_roboflow/new_train/image_161_png.rf.cfd20b4fe1695e91f26646da22150381.jpg
dataset_roboflow/new_train/image_97_png.rf.82f9aba938949bedc9318178f2d42292.jpg
dataset_roboflow/train/image_36_png.rf.cf8a6d8e41936d90a3ea6d59052cc4e5.jpg
dataset_roboflow/new_train/image_91_png.rf.6153c5683671b88991c985917573bc58.jpg
dataset_roboflow/train/image_182_png.rf.6a6b8609bedf8aedb3fb4208b3231579.jpg
dataset_roboflow/new_train/image_108_png.rf.81a586ab7642048a8fe556320feb4af2.jpg
dataset_roboflow/train/image_55_png.rf.3f895c00b966af76d1057ee8f2218ace.jpg
dataset_roboflow/train/image_107_png.rf.f48cbcdeaf88cb6aad0a97358d70e81c.jpg
dataset_roboflow/train/image_200_png.rf.35c299086ff7729f94fae9568a5748

0

In [5]:
new_dataset_path=f'{path}/new_dataset/'


In [6]:
THRES_SCORE = 0.4

def draw_detections(image, box, scores=0, label=''):
#   for box, score, label in zip(boxes[0], scores[0], labels[0]):
#     if score < THRES_SCORE:
#         continue

#     color = label_color(labels)

#     box = box.astype(int)
    
    draw_box(image, box, color=(0,255,0) )

    caption = "{} {:.3f}".format(label, 0)
    draw_caption(image, box, caption)
    
    
    

def show_detected_objects(image_row,img_folder=''):
    
    img_path = img_folder+image_row.filename
    true_box = [
        image_row.xmin, image_row.ymin, image_row.xmax, image_row.ymax]
    image = read_image_bgr(img_path)

    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    image = preprocess_image(image)
    image, scale = resize_image(image)

#     boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis = 0))
#     print('scale',scale)
#     print('scores',scores)
#     print('labels',labels)
#     print('image',image)

#     boxes /= scale
#     print('boxes',boxes[0])

    draw_box(draw, true_box, color=(255, 0, 0))
    draw_detections(image, box=true_box, label= labels_to_name)
    
#     caption = "{} {:.3f}".format(image_row.class_name,0)
#     draw_caption(image,true_box , image_row.class_name)

    plt.axis('off')
    plt.imshow(draw)
    plt.show()

In [7]:
roboflow = pd.read_csv(f'{path}/dataset_roboflow/new_train/annotations.csv')
sketch_it = pd.read_csv(f'{path}/dataset_sketch_it/images/annotations.csv')

In [8]:
roboflow

,image_name,width,height,class_name,x_min,y_min,x_max,y_max
0,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Select,95,98,345,157
1,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Select,102,244,342,309
2,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Button,106,542,372,614
3,00027_png.rf.051c242efd7c63dbc96a34fdfc9e8499.jpg,640,640,Image,12,160,139,400
4,00027_png.rf.051c242efd7c63dbc96a34fdfc9e8499.jpg,640,640,Image,191,151,306,395
...,...,...,...,...,...,...,...,...
4577,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,44,288,621,415
4578,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,27,157,257,275
4579,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,277,153,620,269
4580,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,319,34,607,133


In [9]:
sketch_it

,image_name,width,height,class_name,x_min,y_min,x_max,y_max
0,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,423,332,690,414
1,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,Button,989,422,1186,554
2,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,Label,18,146,328,207
3,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,989,220,1292,294
4,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,RadioButton,804,107,952,168
...,...,...,...,...,...,...,...,...
1996,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,453,628,645,678
1997,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Paragraph,133,734,369,855
1998,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Paragraph,300,204,1072,478
1999,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,752,639,983,679


In [10]:
new_dataset = pd.concat([sketch_it,roboflow],ignore_index=True)
new_dataset

,image_name,width,height,class_name,x_min,y_min,x_max,y_max
0,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,423,332,690,414
1,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,Button,989,422,1186,554
2,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,Label,18,146,328,207
3,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,989,220,1292,294
4,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,RadioButton,804,107,952,168
...,...,...,...,...,...,...,...,...
6578,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,44,288,621,415
6579,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,27,157,257,275
6580,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,277,153,620,269
6581,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,319,34,607,133


In [11]:
new_dataset.rename(columns={'image_name': 'filename'}, inplace=True)
new_dataset.rename(columns={'x_min': 'xmin'}, inplace=True)
new_dataset.rename(columns={'y_min': 'ymin'}, inplace=True)
new_dataset.rename(columns={'x_max': 'xmax'}, inplace=True)
new_dataset.rename(columns={'y_max': 'ymax'}, inplace=True)
new_dataset.rename(columns={'class_name': 'label'}, inplace=True)


roboflow.rename(columns={'image_name': 'filename'}, inplace=True)
roboflow.rename(columns={'x_min': 'xmin'}, inplace=True)
roboflow.rename(columns={'y_min': 'ymin'}, inplace=True)
roboflow.rename(columns={'x_max': 'xmax'}, inplace=True)
roboflow.rename(columns={'y_max': 'ymax'}, inplace=True)
roboflow.rename(columns={'class_name': 'label'}, inplace=True)

sketch_it.rename(columns={'image_name': 'filename'}, inplace=True)
sketch_it.rename(columns={'x_min': 'xmin'}, inplace=True)
sketch_it.rename(columns={'y_min': 'ymin'}, inplace=True)
sketch_it.rename(columns={'x_max': 'xmax'}, inplace=True)
sketch_it.rename(columns={'y_max': 'ymax'}, inplace=True)
sketch_it.rename(columns={'class_name': 'label'}, inplace=True)

new_dataset

,filename,width,height,label,xmin,ymin,xmax,ymax
0,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,423,332,690,414
1,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,Button,989,422,1186,554
2,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,Label,18,146,328,207
3,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,989,220,1292,294
4,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,RadioButton,804,107,952,168
...,...,...,...,...,...,...,...,...
6578,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,44,288,621,415
6579,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,27,157,257,275
6580,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,277,153,620,269
6581,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,319,34,607,133


In [12]:


def copy2dri(df,destination,fromDir=[]):

    print(destination)

    for i in df.filename: 


    #     create dir
        os.makedirs(destination,exist_ok=True)

        for Dir in fromDir:
#             print(f'{Dir+i} -')
            
            if os.path.exists(Dir+i):
                pli_img = Image.open(Dir+i)  
                cv_img = cv2.imread(Dir+i)

#                 print(f'{Dir+i} - {pli_img.format}')
        #         cv2.imwrite(destination+'/'+i, cv_img, [int(cv2.IMWRITE_JPEG_QUALITY), 100])  
                shutil.copy2(Dir+i,destination)


    df.to_csv(destination+'/annotations.csv',index=False)      

    return 'done'

# filter by accpted classes

In [13]:
labels_to_name=pd.DataFrame( new_dataset['label'].unique()).sort_values(by=0)
labels_to_name

,0
1,Button
12,Carousel
0,CheckBox
6,ComboBox
5,Heading
8,Image
2,Label
7,Link
14,Pagination
9,Paragraph


In [14]:
new_dataset.label.value_counts()

label
Label          1075
TextBox         719
CheckBox        695
Button          643
Image           592
Link            491
Heading         490
RadioButton     416
Paragraph       360
Select          351
Table           214
Carousel        178
Textarea        151
Pagination      150
ComboBox         58
Name: count, dtype: int64

In [15]:
accpted_classes = pd.read_csv(f'{path}/accpted_classes.csv')
accpted_classes

,class_label
0,Button
1,CheckBox
2,Heading
3,Image
4,Label
5,Link
6,Paragraph
7,RadioButton
8,Select
9,TextBox


In [16]:
accpted_classes_list = accpted_classes.values.reshape(1,-1)[0]

classes = []

for i,item in enumerate(accpted_classes_list):
        classes.append({'name':item, 'id':i+1})


with open(f'{path}/label_map.pbtxt', 'w') as f:
    for item in classes:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(item['name']))
        f.write('\tid:{}\n'.format(item['id']))
        f.write('}\n') 

In [17]:
new_dataset = new_dataset[new_dataset.label.isin(list(accpted_classes.values.reshape(1,-1)[0]))]
new_dataset

,filename,width,height,label,xmin,ymin,xmax,ymax
0,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,423,332,690,414
1,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,Button,989,422,1186,554
2,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,Label,18,146,328,207
3,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,989,220,1292,294
4,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,RadioButton,804,107,952,168
...,...,...,...,...,...,...,...,...
6569,image_65_png.rf.e3218cec368b8116517298850b15d9...,640,640,CheckBox,393,125,563,149
6570,image_65_png.rf.e3218cec368b8116517298850b15d9...,640,640,CheckBox,401,161,565,182
6571,image_65_png.rf.e3218cec368b8116517298850b15d9...,640,640,CheckBox,395,194,573,220
6572,image_65_png.rf.e3218cec368b8116517298850b15d9...,640,640,CheckBox,201,122,340,140


In [18]:

copy2dri(destination=new_dataset_path,df= new_dataset,fromDir=[f'{path}/dataset_roboflow/new_train/',f'{path}/dataset_sketch_it/images/'])


./new_dataset/


'done'

In [19]:
new_dataset = pd.read_csv(f'{path}/new_dataset/annotations.csv')
new_dataset

,filename,width,height,label,xmin,ymin,xmax,ymax
0,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,423,332,690,414
1,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,Button,989,422,1186,554
2,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,Label,18,146,328,207
3,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,989,220,1292,294
4,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,RadioButton,804,107,952,168
...,...,...,...,...,...,...,...,...
5827,image_65_png.rf.e3218cec368b8116517298850b15d9...,640,640,CheckBox,393,125,563,149
5828,image_65_png.rf.e3218cec368b8116517298850b15d9...,640,640,CheckBox,401,161,565,182
5829,image_65_png.rf.e3218cec368b8116517298850b15d9...,640,640,CheckBox,395,194,573,220
5830,image_65_png.rf.e3218cec368b8116517298850b15d9...,640,640,CheckBox,201,122,340,140


In [20]:
new_dataset[new_dataset['filename']=='8ae99b35-2354-4f8b-ade0-9fd98f09f4fe.png']

,filename,width,height,label,xmin,ymin,xmax,ymax
615,8ae99b35-2354-4f8b-ade0-9fd98f09f4fe.png,900,1248,RadioButton,154,1047,384,1140
616,8ae99b35-2354-4f8b-ade0-9fd98f09f4fe.png,900,1248,Label,207,756,579,872
617,8ae99b35-2354-4f8b-ade0-9fd98f09f4fe.png,900,1248,Label,126,113,518,204
618,8ae99b35-2354-4f8b-ade0-9fd98f09f4fe.png,900,1248,Label,207,946,549,1044
619,8ae99b35-2354-4f8b-ade0-9fd98f09f4fe.png,900,1248,RadioButton,154,870,453,958
620,8ae99b35-2354-4f8b-ade0-9fd98f09f4fe.png,900,1248,Label,167,439,374,522
621,8ae99b35-2354-4f8b-ade0-9fd98f09f4fe.png,900,1248,Heading,134,9,379,84
622,8ae99b35-2354-4f8b-ade0-9fd98f09f4fe.png,900,1248,Label,162,320,420,385
623,8ae99b35-2354-4f8b-ade0-9fd98f09f4fe.png,900,1248,TextBox,446,209,760,305
624,8ae99b35-2354-4f8b-ade0-9fd98f09f4fe.png,900,1248,RadioButton,152,695,415,776


# spilt into train and test

In [21]:
new_dataset.label.value_counts()

label
Label          1075
TextBox         719
CheckBox        695
Button          643
Image           592
Link            491
Heading         490
RadioButton     416
Paragraph       360
Select          351
Name: count, dtype: int64

In [22]:


if os.path.exists('train'):
    shutil.rmtree('train')
    
if os.path.exists('test'):
    shutil.rmtree('test')

train_set = pd.DataFrame([])
test_set = pd.DataFrame([])
lbls = new_dataset.label.unique().tolist()

for lbl in lbls:

    train, test = train_test_split(new_dataset[new_dataset['label']==lbl], test_size = 0.2,shuffle=False)
    train_set=pd.concat([train_set,train],ignore_index=True)
    test_set=pd.concat([test_set,test],ignore_index=True)
    

In [23]:
train_set

,filename,width,height,label,xmin,ymin,xmax,ymax
0,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,423,332,690,414
1,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,989,220,1292,294
2,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,419,243,658,304
3,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,1329,297,1580,372
4,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,698,227,948,302
...,...,...,...,...,...,...,...,...
4657,00019_png.rf.30acb091fbd4f5ec416af77dc9445b91.jpg,640,640,Select,75,432,209,500
4658,00019_png.rf.30acb091fbd4f5ec416af77dc9445b91.jpg,640,640,Select,74,529,211,598
4659,00005_png.rf.59ae759d32184862d21e8e47699a3f75.jpg,640,640,Select,105,480,205,517
4660,00036_png.rf.2b34e2144c9bcae70ffe42d5ea58dad5.jpg,640,640,Select,35,86,570,143


In [24]:
test_set

,filename,width,height,label,xmin,ymin,xmax,ymax
0,image_199_png.rf.4ad9745c6688cf1585a112f4e263d...,640,640,CheckBox,485,560,620,585
1,image_74_png.rf.52beed189736dbe45515e7820607e6...,640,640,CheckBox,473,226,558,246
2,image_74_png.rf.52beed189736dbe45515e7820607e6...,640,640,CheckBox,472,262,560,283
3,image_74_png.rf.52beed189736dbe45515e7820607e6...,640,640,CheckBox,473,297,559,314
4,image_74_png.rf.52beed189736dbe45515e7820607e6...,640,640,CheckBox,473,334,560,352
...,...,...,...,...,...,...,...,...
1165,image_92_png.rf.5826277764057ab0eb4c61c1fd848b...,640,640,Select,310,574,465,616
1166,image_92_png.rf.5826277764057ab0eb4c61c1fd848b...,640,640,Select,41,565,189,608
1167,image_92_png.rf.5826277764057ab0eb4c61c1fd848b...,640,640,Select,61,462,218,499
1168,image_92_png.rf.5826277764057ab0eb4c61c1fd848b...,640,640,Select,82,370,236,414


In [25]:
copy2dri(destination=f'{path}/train/images',df= train_set,fromDir=[new_dataset_path])


./train/images


'done'

In [26]:
copy2dri(destination=f'{path}/test/images',df= test_set,fromDir=[new_dataset_path])


./test/images


'done'

In [27]:
test_set = pd.read_csv(f'{path}/test/images/annotations.csv')
train_set = pd.read_csv(f'{path}/train/images/annotations.csv')

# CSV to coco format

In [28]:
#coco


def csv2coco( data ,save_json_path):

#     data = new_dataset.copy()

    images = []
    categories = []
    annotations = []

    category = {}
    category["supercategory"] = 'none'
    category["id"] = 0
    category["name"] = 'None'
    categories.append(category)

    data['fileid'] = data['filename'].astype('category').cat.codes
    data['categoryid']= pd.Categorical(data['label'],ordered= True).codes
    data['categoryid'] = data['categoryid']+1
    data['annid'] = data.index

    def image(row):
        image = {}
        image["height"] = row.height
        image["width"] = row.width
        image["id"] = row.fileid
        image["file_name"] = row.filename
        return image

    def category(row):
        category = {}
        category["supercategory"] = 'None'
        category["id"] = row.categoryid
        category["name"] = row.label
        return category

    def annotation(row):
        annotation = {}
        area = (row.xmax -row.xmin)*(row.ymax - row.ymin)
        annotation["segmentation"] = []
        annotation["iscrowd"] = 0
        annotation["area"] = area
        annotation["image_id"] = row.fileid

        annotation["bbox"] = [row.xmin, row.ymin, row.xmax -row.xmin,row.ymax-row.ymin ]

        annotation["category_id"] = row.categoryid
        annotation["id"] = row.annid
        return annotation

    for row in data.itertuples():
        annotations.append(annotation(row))

    imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
    for row in imagedf.itertuples():
        images.append(image(row))

    catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
    for row in catdf.itertuples():
        categories.append(category(row))

    data_coco = {}
    data_coco["images"] = images
    data_coco["categories"] = categories
    data_coco["annotations"] = annotations


    json.dump(data_coco, open(save_json_path, "w"), indent=4)

In [29]:
DATA_SET_NAME = f'{path}/new_dataset/'
ANNOTATIONS_FILE_NAME = "annotations.coco.json"

In [30]:
csv2coco(new_dataset.copy(),f'{path}/new_dataset/{ANNOTATIONS_FILE_NAME}')

In [31]:
csv2coco(train_set.copy(),f'{path}/train/images/{ANNOTATIONS_FILE_NAME}')

In [32]:
csv2coco(test_set.copy(),f'{path}/test/images/{ANNOTATIONS_FILE_NAME}')

# CSV to yolo format

In [33]:
labels = {
    "train":os.path.join(f'{path}/train',"labels"),
    "test":os.path.join(f'{path}/test',"labels"),
    
}
for label in labels.values():
    os.makedirs(label,exist_ok=True)

In [34]:
def csv_2_yolo(df,out_path,cocoAnnPath):
    f = open(cocoAnnPath)
    data = json.load(f)
    f.close()
    print(data['categories'])

    def get_img_ann(image_id):
        img_ann = []
        isFound = False
        for ann in data['annotations']:
            if ann['image_id'] == image_id:
                img_ann.append(ann)
                isFound = True
        if isFound:
            return img_ann
        else:
            return None


    def get_img(filename):
        for img in data['images']:
            if img['file_name'] == filename:
                return img
        
    
    for filename in df['filename']:
        # Extracting image 
        img = get_img(filename)
        img_id = img['id']
        img_w = img['width']
        img_h = img['height']

        # Get Annotations for this image
        img_ann = get_img_ann(img_id)

        if img_ann:
        # Opening file for current image
            file_object = open(f"{out_path}/{filename[:-4]}.txt", "a")

        for ann in img_ann:
            current_category = ann['category_id'] - 1 # As yolo format labels start from 0 
            current_bbox = ann['bbox']
            x = current_bbox[0]
            y = current_bbox[1]
            w = current_bbox[2]
            h = current_bbox[3]

            # Finding midpoints
            x_centre = (x + (x+w))/2
            y_centre = (y + (y+h))/2

            # Normalization
            x_centre = x_centre / img_w
            y_centre = y_centre / img_h
            w = w / img_w
            h = h / img_h

            # Limiting upto fix number of decimal places
            x_centre = format(x_centre, '.6f')
            y_centre = format(y_centre, '.6f')
            w = format(w, '.6f')
            h = format(h, '.6f')

            # Writing current object 
            file_object.write(f"{current_category} {x_centre} {y_centre} {w} {h}\n")

        file_object.close()
    print("done")


In [35]:
train = pd.read_csv(f'{path}/train/images/annotations.csv')
test = pd.read_csv(f'{path}/test/images/annotations.csv')

csv_2_yolo(df=train,cocoAnnPath=f'{path}/train/images/annotations.coco.json',out_path=labels['train'])
csv_2_yolo(df=test,cocoAnnPath=f'{path}/test/images/annotations.coco.json',out_path=labels['test'])



[{'supercategory': 'none', 'id': 0, 'name': 'None'}, {'supercategory': 'None', 'id': 1, 'name': 'Button'}, {'supercategory': 'None', 'id': 2, 'name': 'CheckBox'}, {'supercategory': 'None', 'id': 3, 'name': 'Heading'}, {'supercategory': 'None', 'id': 4, 'name': 'Image'}, {'supercategory': 'None', 'id': 5, 'name': 'Label'}, {'supercategory': 'None', 'id': 6, 'name': 'Link'}, {'supercategory': 'None', 'id': 7, 'name': 'Paragraph'}, {'supercategory': 'None', 'id': 8, 'name': 'RadioButton'}, {'supercategory': 'None', 'id': 9, 'name': 'Select'}, {'supercategory': 'None', 'id': 10, 'name': 'TextBox'}]
done
[{'supercategory': 'none', 'id': 0, 'name': 'None'}, {'supercategory': 'None', 'id': 1, 'name': 'Button'}, {'supercategory': 'None', 'id': 2, 'name': 'CheckBox'}, {'supercategory': 'None', 'id': 3, 'name': 'Heading'}, {'supercategory': 'None', 'id': 4, 'name': 'Image'}, {'supercategory': 'None', 'id': 5, 'name': 'Label'}, {'supercategory': 'None', 'id': 6, 'name': 'Link'}, {'supercategory':

In [36]:

#all img in yolo formate

all_yolo_path = {
    "all_img":os.path.join(f'{path}/all_yolo',"images"),
    "all_label":os.path.join(f'{path}/all_yolo',"labels"),
    
}

for _path in all_yolo_path.values():
    os.makedirs(_path,exist_ok=True)

all_yolo = pd.read_csv(f'{path}/new_dataset/annotations.csv')

copy2dri(destination=all_yolo_path["all_img"],df=all_yolo,fromDir=[new_dataset_path])
csv_2_yolo(df=all_yolo,cocoAnnPath=f'{path}/new_dataset/annotations.coco.json',out_path=all_yolo_path['all_label'])


./all_yolo/images
[{'supercategory': 'none', 'id': 0, 'name': 'None'}, {'supercategory': 'None', 'id': 1, 'name': 'Button'}, {'supercategory': 'None', 'id': 2, 'name': 'CheckBox'}, {'supercategory': 'None', 'id': 3, 'name': 'Heading'}, {'supercategory': 'None', 'id': 4, 'name': 'Image'}, {'supercategory': 'None', 'id': 5, 'name': 'Label'}, {'supercategory': 'None', 'id': 6, 'name': 'Link'}, {'supercategory': 'None', 'id': 7, 'name': 'Paragraph'}, {'supercategory': 'None', 'id': 8, 'name': 'RadioButton'}, {'supercategory': 'None', 'id': 9, 'name': 'Select'}, {'supercategory': 'None', 'id': 10, 'name': 'TextBox'}]
done
